# L5: Automate Event Planning

In this lesson, you will learn more about Tasks.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

import sys
import os
sys.path.append(os.path.abspath('..'))

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew
from langchain.llms import Ollama
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

True

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
def get_local_llm():
    print("Loading Ollama")
    return Ollama(model="llama3.1")

def get_azure_llm():
    os.environ["OPENAI_API_VERSION"] = os.getenv("OPENAI_API_VERSION")
    os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
    os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
    print("Loading Azure OpenAI")
    return AzureChatOpenAI(
        deployment_name=os.getenv("AZURE_DEPLOYMENT__GPT_4"),
    )

def get_llm(run_local):
    if run_local == "True" or run_local == "true":
        return get_local_llm()
    else:
        return get_azure_llm()


run_local = os.getenv("RUN_LOCAL")
llm = get_llm(run_local)

Loading Azure OpenAI


In [4]:
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

## crewAI Tools

In [5]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [6]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    ),
    llm=llm
)

In [7]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    ),
    llm=llm
)

In [8]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    ),
    llm=llm
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [9]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [10]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [11]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [12]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [13]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [16]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "Manchester, United Kingdom",
    'tentative_date': "2024-12-10",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note 1**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

**Note 2**: 
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [17]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in Manchester, United Kingdom that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
Thought: I need to find a venue in Manchester, United Kingdom that can accommodate a Tech Innovation Conference. I should start by searching for venues that are suitable for such an event.

Action: Search the internet
Action Input: {"search_query": "Tech Innovation Conference venues in Manchester UK"} 


Search results: Title: Tech conference events in Manchester, United Kingdom - Eventbrite
Link: https://www.eventbrite.com/d/united-kingdom--manchester/tech-conference/
Snippet: Women in Tech Workshop & Networking Event. Wed, Nov 27 • 4:30 PM. InterCity Technology, Hallmark House, Paragon Business Park, Horwich, Bolton BL6 6HG.
---
Title: IT & Technology Events in Manchester - 10Times
Link: https://10times.com/manchester-uk/technology
Snippet: Explore a diverse array of IT & Tec



> Entering new CrewAgentExecutor chain...


> Entering new CrewAgentExecutor chain...
Thought: To effectively market the Tech Innovation Conference and engage potential attendees, I need to develop a marketing strategy that utilizes various channels. First, I should research the most effective platforms and promotional strategies for tech-related events to understand where and how to focus our efforts.

Action: Search the internet
Action Input: {"search_query": "most effective marketing strategies for tech conferences"} 


Search results: Title: 12 Effective Conference Marketing Ideas That Drive Registrations
Link: https://memberclicks.com/blog/conference-marketing/
Snippet: Start with a strategy; Build a strong website and event app; Make registration easy; Choose good social channels; Offer incentives; Analyze data.
---
Title: 15 conference marketing ideas for 2024 that really work | Kaltura
Link: https://corp.kaltura.com/blog/conference-marketing/
Snippet: Explore conference marke

- Display the generated `venue_details.json` file.

In [18]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'Manchester, UK',
 'booking_status': 'Available',
 'capacity': 0,
 'name': 'Manchester Central Convention Complex'}


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

In [21]:
from IPython.display import Markdown
Markdown("marketing_report.md")

## Marketing Strategy for the Tech Innovation Conference

### 1. Foundation of Marketing Strategy
- **Start with a Strategy**: Define clear goals, identify the target audience, and plan the marketing activities timeline.
- **Strong Website and Event App**: Develop a user-friendly website and event app to serve as the central information hub.
- **Simplified Registration Process**: Ensure the registration process is straightforward and easy to navigate.
- **Optimal Social Media Channels**: Choose social media platforms where tech enthusiasts and professionals are most active.
- **Incentives**: Offer early bird discounts, exclusive content access, or prize draws to encourage early sign-ups.
- **Data Analysis**: Use tools to track engagement and effectiveness of different marketing tactics, adjusting strategies as needed.

### 2. Engaging Marketing Activities
- **Polls and Surveys**: Engage potential attendees by involving them in decisions through online polls and surveys regarding conference content and features.
- **Interesting Venue**: Promote Manchester Central Convention Complex as a prime location for tech events, highlighting its facilities and accessibility.
- **Sponsor Collaborations**: Partner with well-known tech companies to increase credibility and attract their followers.
- **Giveaways and Contests**: Organize competitions with tech-related prizes to keep potential attendees interested and engaged.
- **Gamification**: Introduce gamified elements in the pre-event phase, such as quizzes about tech innovations.
- **Live Streaming Sessions**: Use live streaming to showcase key speakers and teaser content about the conference topics.
- **Social Media Campaigns**: Regularly update social media with engaging content, announcements, and interactive posts to maintain interest and excitement.
- **Scarcity Tactics**: Implement limited-time offers or limited-availability VIP tickets to create urgency.
- **Testimonials and Case Studies**: Share experiences and positive reviews from past events to build trust and anticipation.
- **Pre-Event Networking**: Facilitate connections between attendees before the event through social media groups or event apps.
- **Pop-Up Events**: Organize small, local meetups or tech talks to promote the main conference.

### 3. Conclusion
Implementing these strategies will not only boost registrations but also create a buzz around the Tech Innovation Conference, making it a must-attend event for tech professionals and enthusiasts. By focusing on a mix of traditional and innovative marketing tactics, we aim to fully engage potential attendees and exceed our engagement goals.

In [22]:
with open('venue_details.json') as f:
    line = f.readlines()
    print(data)

{'name': 'Manchester Central Convention Complex', 'address': 'Manchester, UK', 'capacity': 0, 'booking_status': 'Available'}
